# NOTEBOOK PARTE 2: EDA TRADUCTOR PT -> ES

# CONFIGURACIÓN DEL ENTORNO

## Librerías

In [14]:
import os
from datasets import Dataset
from transformers import AutoTokenizer
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import random
from tqdm.auto import tqdm
from transformers import set_seed

In [15]:
nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jpmon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jpmon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Reproducibilidad

In [16]:
SEED = 333
set_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

## Path de los datos

In [5]:
DATA_DIR = "./data"
PT_FILE = os.path.join(DATA_DIR, "corpus_pt.txt")
ES_FILE = os.path.join(DATA_DIR, "corpus_es.txt")

assert os.path.isfile(PT_FILE), f"No encontrado {PT_FILE}"
assert os.path.isfile(ES_FILE), f"No encontrado {ES_FILE}"

print(f"Rutas configuradas:\n  PT→ {PT_FILE}\n  ES→ {ES_FILE}")

Rutas configuradas:
  PT→ ./data\corpus_pt.txt
  ES→ ./data\corpus_es.txt


# EDA & Limpieza de los datos

## Carga de los datos

In [6]:
# Rutas (definidas en el paso 0)
pt_path = PT_FILE    # "./data/corpus_pt.txt"
es_path = ES_FILE    # "./data/corpus_es.txt"

# Leer líneas
with open(pt_path, encoding="utf-8") as f:
    pt_lines = [line.strip() for line in f]
with open(es_path, encoding="utf-8") as f:
    es_lines = [line.strip() for line in f]

assert len(pt_lines) == len(es_lines), "Los archivos PT y ES no tienen misma cantidad de líneas"
print(f"Pares totales originales: {len(pt_lines)}")

Pares totales originales: 6207844


## Funciones de limpieza

In [7]:
STOP_ES = set(stopwords.words("spanish"))
STOP_PT = set(stopwords.words("portuguese"))

def normalize_spaces(text):
    """Quita espacios y tabs múltiples, retornos de carro."""
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def simple_tokenize(text):
    """Tokeniza con NLTK (útil para EDA de longitud)."""
    return word_tokenize(text)

def is_valid_pair(pt, es, 
                  min_len=2, max_len=256):
    """
    Criterio de filtrado:
      - No vacío
      - Longitud en tokens entre min y max (inclusive)
      - Texto PT y ES no idéntico
    """
    if not pt or not es:
        return False
    t_pt = simple_tokenize(pt)
    t_es = simple_tokenize(es)
    if len(t_pt) < min_len or len(t_es) < min_len:
        return False
    if len(t_pt) > max_len or len(t_es) > max_len:
        return False
    if pt == es:
        return False
    return True

## Estadísticas iniciales

In [8]:
# Estadísticas iniciales (rápido + tqdm)
def fast_len(text: str) -> int:
    """Cuenta tokens con .split(), muy rápido."""
    return len(text.split())

# Longitudes rápidas con tqdm
tqdm.pandas()  # habilita progress_apply si quieres
pt_lens = [fast_len(s) for s in tqdm(pt_lines, desc="Len PT rápido")]
es_lens = [fast_len(s) for s in tqdm(es_lines, desc="Len ES rápido")]

def print_stats(name: str, lengths: list):
    print(f"--- {name} ---")
    print(f"Min   : {np.min(lengths)}")
    print(f"10 pct: {np.percentile(lengths, 10)}")
    print(f"Median: {np.median(lengths)}")
    print(f"90 pct: {np.percentile(lengths, 90)}")
    print(f"Max   : {np.max(lengths)}")
    print()

print_stats("Portugués (fast_len)", pt_lens)
print_stats("Español (fast_len)", es_lens)

# Duplicados y vacíos (rápido, sin tqdm)
pairs = list(zip(pt_lines, es_lines))
num_dups  = len(pairs) - len(set(pairs))
num_empty = sum(1 for p, e in pairs if not p or not e)

print(f"Pares duplicados:      {num_dups}")
print(f"Pares con vacíos:      {num_empty}\n")

Len PT rápido:   0%|          | 0/6207844 [00:00<?, ?it/s]

Len ES rápido:   0%|          | 0/6207844 [00:00<?, ?it/s]

--- Portugués (fast_len) ---
Min   : 0
10 pct: 1.0
Median: 4.0
90 pct: 39.0
Max   : 9152

--- Español (fast_len) ---
Min   : 0
10 pct: 1.0
Median: 5.0
90 pct: 42.0
Max   : 9682

Pares duplicados:      3459848
Pares con vacíos:      16



In [10]:
# Estadística “exacta” en submuestra con NLTK
sample_n = 200_000
idxs = random.sample(range(len(pt_lines)), sample_n)
pt_sample = [pt_lines[i] for i in idxs]
es_sample = [es_lines[i] for i in idxs]

pt_lens_s = [len(simple_tokenize(s)) for s in tqdm(pt_sample, desc="Len PT NLTK sample")]
es_lens_s = [len(simple_tokenize(s)) for s in tqdm(es_sample, desc="Len ES NLTK sample")]

print_stats("PT (NLTK sample)", pt_lens_s)
print_stats("ES (NLTK sample)", es_lens_s)

Len PT NLTK sample:   0%|          | 0/200000 [00:00<?, ?it/s]

Len ES NLTK sample:   0%|          | 0/200000 [00:00<?, ?it/s]

--- PT (NLTK sample) ---
Min   : 1
10 pct: 1.0
Median: 5.0
90 pct: 43.0
Max   : 7356

--- ES (NLTK sample) ---
Min   : 1
10 pct: 1.0
Median: 5.0
90 pct: 46.0
Max   : 1618



## Filtrado de ruido

In [11]:
# Crear set para eliminar duplicados y vacíos de forma eficiente
unique_pairs = set()
clean_pairs = []

for pt, es in tqdm(zip(pt_lines, es_lines), total=len(pt_lines), desc="Filtrando pares"):
    # Normalizar espacios
    pt_n = normalize_spaces(pt)
    es_n = normalize_spaces(es)
    # Salta vacíos
    if not pt_n or not es_n:
        continue
    # Usa el tupla para detectar duplicados
    pair = (pt_n, es_n)
    if pair in unique_pairs:
        continue
    unique_pairs.add(pair)
    # Aplica criterios de validación
    if is_valid_pair(pt_n, es_n, min_len=2, max_len=256):
        clean_pairs.append(pair)

# Se descomprime en dos listas PT y ES
pt_clean, es_clean = zip(*clean_pairs)

# Reporte
total_original = len(pt_lines)
total_filtered = len(clean_pairs)
num_removed = total_original - total_filtered

print(f"Pares originales      : {total_original}")
print(f"Pares tras filtrado   : {total_filtered}")
print(f"Eliminados            : {num_removed}")
print(f"% eliminado           : {num_removed/total_original*100:.2f}%")

Filtrando pares:   0%|          | 0/6207844 [00:00<?, ?it/s]

Pares originales      : 6207844
Pares tras filtrado   : 2480081
Eliminados            : 3727763
% eliminado           : 60.05%


## Construir Dataset final

In [13]:
# Guardar archivos de texto
with open(os.path.join(DATA_DIR, "clean_corpus_pt.txt"), "w", encoding="utf-8") as f_pt, \
     open(os.path.join(DATA_DIR, "clean_corpus_es.txt"), "w", encoding="utf-8") as f_es:
    for pt, es in clean_pairs:
        f_pt.write(pt + "\n")
        f_es.write(es + "\n")

# Guardar como Dataset JSON (opción Hugging Face)
clean_ds = Dataset.from_dict({"pt": list(pt_clean), "es": list(es_clean)})
clean_ds.to_json(os.path.join(DATA_DIR, "clean_corpus_pt_es.json"))

print("Corpus limpio guardado en:")
print("   •", os.path.join(DATA_DIR, "clean_corpus_pt.txt"))
print("   •", os.path.join(DATA_DIR, "clean_corpus_es.txt"))
print("   •", os.path.join(DATA_DIR, "clean_corpus_pt_es.json"))

Creating json from Arrow format:   0%|          | 0/2481 [00:00<?, ?ba/s]

Corpus limpio guardado en:
   • ./data\clean_corpus_pt.txt
   • ./data\clean_corpus_es.txt
   • ./data\clean_corpus_pt_es.json


# Comentarios 

- Calidad y cobertura del corpus:
Tras extraer y alinear las de 6 millones de oraciones del Parlamento Europeo, se comprobó una amplia cobertura temática y un rango muy variado de longitudes (de 1 a casi 10,000 tokens). Esto confconfirmó que se cuenta con un recurso suficientemente grande, aunque heterogéneo, para entrenar un traductor PT→ES robusto.

- Reducción de ruido y duplicados:
El filtrado inicial eliminó más del 60% de los pares, principalmente por duplicados exactos y longitudes extremas. Aunque parezca un recorte drástico, quedarnos con ~2.5 M de pares de calidad media/alta facilitará el entrenamiento y evitará que el modelo “memorice” fragmentos repetidos o aprenda de ejemplos demasiado largos o irrelevantes.

- Elección de umbrales y criterios de limpieza:
Definimos como válidas las oraciones con entre 2 y 256 tokens, y descartamos pares idénticos. Estos límites balancean la necesidad de capturar construcciones complejas contra la eficiencia de entrenamiento y la memoria. En futuros refinamientos podríamos ajustar estos umbrales o añadir chequeos como la relación de longitud PT/ES o la eliminación de oraciones con demasiados números o símbolos.

- Modularidad y reproducibilidad del flujo:
Se decidió dividir el pipeline en dos notebooks —uno para limpieza y otro para modelado— para obtener flexibilidad: con esto podemos iterar en los criterios de EDA sin reentrenar, y lanzar rápidamente nuevos experimentos de fine-tuning usando el corpus limpio ya almacenado. Además, los archivos planos (.txt) garantizan compatibilidad con cualquier script o framework.

- Próximos pasos:
Con el conjunto limpio listo, el siguiente paso será la división en train/test y la tokenización específica para el modelo. Será importante mantener la semilla y la misma lógica de particionado para asegurar que las comparaciones de configuraciones de entrenamiento sean consistentes y reproducibles.